In [ ]:
import chainlit as cl
import os
import asyncio
import threading
import requests
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate

# Set up Ollama (you may need to run this in a separate cell)
!sudo apt-get install -y pciutils lshw
!curl -fsSL https://ollama.com/install.sh | sh

# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'
# Set the environment variable

# Load configuration
with open('../config.json') as config_file:
    config = json.load(config_file)
    os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Ollama setup functions
async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Start Ollama serve
new_loop = asyncio.new_event_loop()
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()



In [ ]:
# download bggpt model
!ollama pull todorov/bggpt:latest --name False

# Constants and configurations
CHROMA_PATH = "full_books_800"
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}

Answer always in full sentences.
Answer always in Bulgarian.
Reformat the result to be more readable.
Reformat to exclude repetitions of words.
Приказката трябва да бъде дълга и интересна.
Приказката трябва да има поучаващ край.
Приказката не трябва да бъде със съдържание, което не е подходящо за деца.
Приказката трябва да бъде със съдържание, което е подходящо за деца.
Приказката не трябва да съдаържа повторения, като един ден, итн.
"""

# Initialize embedding function and database
embedding_function = OpenAIEmbeddings(model='text-embedding-3-small')
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Function to generate text using Ollama API
def generate_text(prompt):
    url = "http://localhost:11434/api/generate"
    headers = {'Content-Type': 'application/json'}
    data = {
        "model": "todorov/bggpt:latest",
        "stream": False,
        "prompt": prompt
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return json.loads(response.text)["response"]
    else:
        raise Exception(f"Error: {response.status_code} {response.text}")

def main():
    print("Моля, въведете тема за приказката и натиснете Enter.")

    while True:
        query_text = input("Тема: ").strip()

        if not query_text:
            print("Моля, въведете тема преди да продължите.")
            continue

        results = db.similarity_search_with_relevance_scores(query_text, k=1)

        if not results:
            print("Не бяха намерени подходящи резултати. Опитайте с друга тема.")
            continue

        context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
        prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        prompt = prompt_template.format(context=context_text, question=query_text)

        response_text = generate_text(prompt)

        sources = [doc.metadata.get("source", None) for doc, _score in results]
        sources_text = ", ".join(filter(None, sources))

        formatted_response = f"{response_text}\n\nИзточници: {sources_text}"

        print("\n" + formatted_response + "\n")

        if input("Искате ли да генерирате друга приказка? (да/не): ").lower() != 'да':
            break

if __name__ == "__main__":
    main()

In [ ]:
import chainlit as cl
from pyngrok import ngrok
import subprocess
import threading
import IPython
import signal
import os
import sys
from io import StringIO
import time

public_url = None
thread = None
process = None
stop_flag = threading.Event()

# Add these global variables
output_buffer = StringIO()
error_buffer = StringIO()

def run_app():
    global process, output_buffer, error_buffer, stop_flag
    try:
        process = subprocess.Popen(
            ["chainlit", "run", "/content/drive/MyDrive/repos/GenPrikazki/fairy_tale_generator.py", "--host", "0.0.0.0", "--port", "8000"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        if process is None:
            print("Failed to start the Chainlit process. Check if Chainlit is installed and the file path is correct.")
            return

        while not stop_flag.is_set():
            output = process.stdout.readline()
            error = process.stderr.readline()

            if output:
                print(output.strip())
                output_buffer.write(output)
            if error:
                print(f"Error: {error.strip()}", file=sys.stderr)
                error_buffer.write(error)

            if process.poll() is not None:
                print("Chainlit process has ended unexpectedly.")
                break

            time.sleep(0.1)
    except Exception as e:
        print(f"Error in run_app: {e}")
        print(f"Process state: {process}")
        if process:
            print(f"Process return code: {process.returncode}")
    finally:
        if process:
            process.terminate()
            process.wait()

def print_logs():
    global output_buffer, error_buffer
    print("=== Chainlit App Output ===")
    print(output_buffer.getvalue())
    print("\n=== Chainlit App Errors ===")
    print(error_buffer.getvalue())

def start_app():
    global public_url, thread, stop_flag
    stop_flag.clear()

    try:
        # Try to set up ngrok
        try:
            public_url = ngrok.connect(8000)
            print(f"Public URL: {public_url}")
        except exception.PyngrokNgrokError as e:
            print(f"Ngrok error: {e}")
            print("Continuing without ngrok. You can access the app using Colab's public URL feature.")
            public_url = None

        # Run the app in a separate thread
        thread = threading.Thread(target=run_app)
        thread.start()

        if public_url:
            print("Your Chainlit app is now running. Access it using the Public URL provided above.")
        else:
            print("Your Chainlit app is now running. Use Colab's 'Public URL' feature to access it.")
            print("Go to Runtime > Manage Sessions > Public URL to get the URL.")
    except Exception as e:
        print(f"Error starting app: {e}")

def stop_app():
    global public_url, thread, process, stop_flag

    stop_flag.set()

    # Stop the Chainlit process
    if process:
        try:
            os.kill(process.pid, signal.SIGTERM)
        except ProcessLookupError:
            pass
        process = None

    # Stop the thread
    if thread:
        thread.join(timeout=5)
        thread = None

    # Close the ngrok tunnel
    if public_url:
        try:
            ngrok.disconnect(public_url)
        except Exception as e:
            print(f"Error disconnecting ngrok: {e}")
        public_url = None

    print_logs()
    output_buffer.truncate(0)
    output_buffer.seek(0)
    error_buffer.truncate(0)
    error_buffer.seek(0)

    print("Chainlit app has been stopped and ngrok tunnel closed.")

# Start the app
start_app()

# Keep the Colab notebook running
IPython.display.Javascript("""
    function ClickConnect(){
        console.log("Working");
        document.querySelector("colab-connect-button").click()
    }
    setInterval(ClickConnect, 60000)
""")

# To stop the app, run the following function:
# stop_app()

In [ ]:
import psutil
import signal
from pyngrok import ngrok

def stop_running_app():
    # Find and stop the Chainlit process
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            # Check if the process is the Chainlit app
            if 'chainlit' in proc.info['name'] or any('chainlit' in arg for arg in proc.info['cmdline']):
                print(f"Stopping Chainlit process (PID: {proc.info['pid']})")
                os.kill(proc.info['pid'], signal.SIGTERM)
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

    # Close all ngrok tunnels
    try:
        tunnels = ngrok.get_tunnels()
        for tunnel in tunnels:
            print(f"Closing ngrok tunnel: {tunnel.public_url}")
            ngrok.disconnect(tunnel.public_url)
    except Exception as e:
        print(f"Error closing ngrok tunnels: {e}")

    print("Attempted to stop the Chainlit app and close ngrok tunnels.")

# Run the function to stop the app
stop_running_app()